In [ ]:
!nvidia-smi  # GPU 확인

# 1. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

TARGET_DIR = "/content/drive/MyDrive/your_path"

# 2. 클론할 위치로 이동
%cd  $TARGET_DIR

# 3. GitHub 레포 클론
!git clone https://github.com/MeshalAlamr/quick-action-recognition.git

# 4. 프로젝트 폴더로 이동
%cd quick-action-recognition

# 5. 라이브러리 설치
!pip install torch torchvision
!pip install numpy tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install gdown

FILE_ID = "103NOL9YYZSW1hLoWmYnv5Fs8mK-Ij7qb"
ZIP_NAME = "dataset.zip"

!gdown --id {FILE_ID} -O {ZIP_NAME}
!mkdir -p data
!unzip -q {ZIP_NAME} -d data

print("다운로드 & 압축 해제 완료!")


In [ ]:
import numpy as np
import pickle
import gc

# 선택할 액션들
actions = [0, 1, 7, 8, 10, 11, 12, 32, 27, 28, 29, 40]

# 프로젝트 이름(= GitHub repo 이름)
PROJECT_NAME = "quick-action-recognition"

# 자동으로 path 구성
path = f"{TARGET_DIR}/{PROJECT_NAME}/"

# ---- 라벨은 그냥 일반 로딩 (크기 작음) ----
with open("/content/data/data/NTU-RGB-D/xview/train_label.pkl", "rb") as f:
    train_label = pickle.load(f)   # (2, N_train) 구조라고 가정
with open("/content/data/data/NTU-RGB-D/xview/val_label.pkl", "rb") as f:
    val_label = pickle.load(f)     # (2, N_val)

train_labels = np.array(train_label[1])
val_labels   = np.array(val_label[1])

# 선택할 인덱스 구하기
train_index = np.where(np.isin(train_labels, actions))[0]
val_index   = np.where(np.isin(val_labels,   actions))[0]

print("num selected train samples:", len(train_index))
print("num selected val samples  :", len(val_index))

# ---- 데이터는 메모리 매핑으로 로드 (RAM에 전체 안 올림) ----
train_data_mm = np.load("/content/data/data/NTU-RGB-D/xview/train_data.npy", mmap_mode="r")
val_data_mm   = np.load("/content/data/data/NTU-RGB-D/xview/val_data.npy",   mmap_mode="r")

print("train_data shape:", train_data_mm.shape)
print("val_data shape  :", val_data_mm.shape)

# ---- small_train_data / small_val_data를 미리 할당 ----
small_train_shape = (len(train_index),) + train_data_mm.shape[1:]
small_val_shape   = (len(val_index),)   + val_data_mm.shape[1:]

small_train_data = np.empty(small_train_shape, dtype=train_data_mm.dtype)
small_val_data   = np.empty(small_val_shape,   dtype=val_data_mm.dtype)

# ---- 하나씩 복사 (메모리 피크 줄이기) ----
for out_i, src_i in enumerate(train_index):
    small_train_data[out_i] = train_data_mm[src_i]

for out_i, src_i in enumerate(val_index):
    small_val_data[out_i] = val_data_mm[src_i]

# ---- small 데이터 저장 ----
np.save(path + "small_train_data.npy", small_train_data)
np.save(path + "small_val_data.npy",   small_val_data)

# ---- small 라벨도 같이 저장 ----
small_train_label = np.array(train_label)   # (2, N_train)
small_train_label = small_train_label[:, train_index]

small_val_label = np.array(val_label)
small_val_label = small_val_label[:, val_index]

with open(path + "small_train_label.pkl", "wb") as f:
    pickle.dump(small_train_label, f)

with open(path + "small_val_label.pkl", "wb") as f:
    pickle.dump(small_val_label, f)

# 메모리 정리
del train_data_mm, val_data_mm, small_train_data, small_val_data
gc.collect()
print("Done.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 프로젝트 이름(= GitHub repo 이름)
PROJECT_NAME = "quick-action-recognition"

# 자동으로 path 구성
resultsFolder = f"{TARGET_DIR}/{PROJECT_NAME}/"

import os
os.makedirs(resultsFolder, exist_ok=True)

In [ ]:
import numpy as np
import pandas as pd
import torch
import pickle

# 기존 small_train / small_val 로딩 (train은 안 건드려도 됨)
features_train = torch.FloatTensor(np.load(path + "small_train_data.npy"))
features_val   = torch.FloatTensor(np.load(path + "small_val_data.npy"))

# small_val_label은 (2, N) 구조라고 가정
small_val_label = pickle.load(open(path + "small_val_label.pkl", "rb"))  # shape: (2, N)
val_ids    = np.array(small_val_label[0])   # 샘플 id
val_labels = np.array(small_val_label[1])   # 실제 클래스 라벨 (stratify용)

print("train samples:", features_train.shape[0])
print("val(samples in small_val):", features_val.shape[0])
print("ratio (val/train):", features_val.shape[0] / features_train.shape[0])

In [ ]:
from sklearn.model_selection import train_test_split

val_ratio = 0.3
test_ratio = 0.7

num_val_all = features_val.shape[0]
indices_all = np.arange(num_val_all)

val_idx, test_idx = train_test_split(
    indices_all,
    test_size=test_ratio,   # TEST 70%
    random_state=42,
    stratify=val_labels     # 클래스 비율 유지
)

print("Split size → VAL:", len(val_idx), "TEST:", len(test_idx))

# -----------------------------------------
# split된 VAL 저장 (small_subval_*)
# -----------------------------------------

final_val_features = features_val[val_idx]
final_val_ids      = val_ids[val_idx]
final_val_labels   = val_labels[val_idx]

# 라벨을 train과 동일한 (2, N) 구조로 다시 묶기
small_subval_label = np.vstack([final_val_ids, final_val_labels])

np.save(path + "small_val_data.npy", final_val_features.numpy())
with open(path + "small_val_label.pkl", "wb") as f:
    pickle.dump(small_subval_label, f)

# -----------------------------------------
# split된 TEST 저장 (small_test_*)
# -----------------------------------------

final_test_features = features_val[test_idx]
final_test_ids      = val_ids[test_idx]
final_test_labels   = val_labels[test_idx]

small_test_label = np.vstack([final_test_ids, final_test_labels])

np.save(path + "small_test_data.npy", final_test_features.numpy())
with open(path + "small_test_label.pkl", "wb") as f:
    pickle.dump(small_test_label, f)

print("Done: small_val_data/label (VAL) + small_test_data/label (TEST) saved.")